In [1]:
import os
import time
import json
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

def read_shapefiles(mode):
    base_path = 'E:/STP_PWoD/Data/Data_shapefile'
    origin = gpd.read_file(f'{base_path}/Origin_shp/origin_{mode}_shp/origin_{mode}.shp')
    destination = gpd.read_file(f'{base_path}/Destn_shp/destn_{mode}_shp/destn_{mode}.shp')
    ppa = gpd.read_file(f'E:/STP_PWoD/PPA_join/{mode}_PPA_join/{mode}_PPA_join.shp')
    return origin, destination, ppa

def extract_opening_hours(hours_str):
    if pd.notna(hours_str):
        hours_json = json.loads(hours_str)
        weekday_hours = hours_json.get("Tue", [])
        weekend_hours = hours_json.get("Sun", [])
        if weekday_hours:
            return f"[{weekday_hours[0][0]},{weekday_hours[0][1]}]"
        elif weekend_hours:
            return f"[{weekend_hours[0][0]},{weekend_hours[0][1]}]"
    return "[00:00,24:00]"

def add_12_hours(t):
    hours, minutes = t.hour, t.minute
    new_hour = (hours + 12) % 24
    return pd.Timestamp(f"{new_hour:02}:{minutes:02}").time()

def process_poi_data():
    POI_csv = pd.read_csv('E:/STP_PWoD/POI_Utah/POI_Utah.csv', usecols=['placekey','sg_c__location_name','sg_c__longitude','sg_c__latitude','sg_c__open_hours'])
    POI_csv['ser_hr'] = POI_csv['sg_c__open_hours'].apply(extract_opening_hours)
    POI_csv[['opening_hr', 'closing_hr']] = POI_csv['ser_hr'].str.strip('[]').str.split(',', expand=True)
    POI_csv['closing_hr'] = POI_csv['closing_hr'].replace('24:00', '23:59')
    POI_csv['opening_hour'] = pd.to_datetime(POI_csv['opening_hr'], format='%H:%M').dt.time
    POI_csv['closing_hour'] = pd.to_datetime(POI_csv['closing_hr'], format='%H:%M').dt.time
    mask = POI_csv['opening_hour'] > POI_csv['closing_hour']
    POI_csv.loc[mask, 'closing_hour'] = POI_csv.loc[mask, 'closing_hour'].apply(add_12_hours)
    geometry = [Point(xy) for xy in zip(POI_csv['sg_c__longitude'], POI_csv['sg_c__latitude'])]
    POI_all1 = gpd.GeoDataFrame(POI_csv, geometry=geometry).set_crs(epsg=4326, inplace=True)
    return POI_all1

def process_mode_data(mode, POI_all1, shapefiles_dict):
    origin, destination, ppa = shapefiles_dict[mode]
    target_crs = ppa.crs
    POI_all1 = POI_all1.to_crs(target_crs)
    join_PPA_POIall1 = gpd.sjoin(POI_all1, ppa, how='inner', predicate='within')

    join_PPA_POIall1['departure_time_new'] = pd.to_datetime(join_PPA_POIall1['depart_hhm'], format='%H%M', errors='coerce').dt.time
    join_PPA_POIall1['arrival_time_new'] = pd.to_datetime(join_PPA_POIall1['arrive_hhm'], format='%H%M', errors='coerce').dt.time

    FOS_PPA = join_PPA_POIall1[
        (join_PPA_POIall1['departure_time_new'] >= join_PPA_POIall1['opening_hour']) &
        (join_PPA_POIall1['departure_time_new'] <= join_PPA_POIall1['closing_hour'])
    ]

    hptripid_counts = FOS_PPA['hptripid_1'].value_counts()

    FOS_PPA_check = FOS_PPA[
        [
            'hptripid','sg_c__location_name','sg_c__longitude', 'sg_c__latitude','geometry','ser_hr',
            'hhmemberid','depart_hhm', 'o_fixed', 'd_fixed', 'ACRES', 'DEVACRES', 'DEVPBLEPCT','X','Y', 'Shape_Leng', 'Shape_Area'
        ]
    ]
    FOS_PPA_check = FOS_PPA_check.rename(columns={
        'sg_c__location_name':'loc_name',
        'sg_c__longitude':'long',
        'sg_c__latitude':'lat'
    })

    unique_hptripids = FOS_PPA_check['hptripid'].unique()

    for hptripid in unique_hptripids:
        filtered_df = FOS_PPA_check[FOS_PPA_check['hptripid'] == hptripid]
        geometry = [Point(xy) for xy in zip(filtered_df['long'], filtered_df['lat'])]
        geo_df = gpd.GeoDataFrame(filtered_df, geometry=geometry)
        geo_df.crs = "EPSG:4326"
        filename = f"POI_{mode}_{hptripid.replace('.', '_')}.shp"
        filepath = os.path.join(f"E:/STP_PWoD/POI_after_Ser_hr_extracted/{mode}_POI_after_ser_hr", filename)
        geo_df.to_file(filepath, driver='ESRI Shapefile')
        
    return hptripid_counts, FOS_PPA

def main():
    start_time = time.time()

    modes = ['Bike']  #['Auto', 'Bike', 'Transit', 'Walk']
    shapefiles_dict = {mode: read_shapefiles(mode) for mode in modes}
    POI_all1 = process_poi_data()

    for mode in modes:
        hptripid_counts, FOS_PPA = process_mode_data(mode, POI_all1, shapefiles_dict)
        for hptripid, count in hptripid_counts.items():
            print(f"For hptripid polygon in {mode} mode: {hptripid}, POI count= {count}")
        print(f'Size of Filtered dataset for {mode} mode is:', FOS_PPA.shape[0])

    end_time = time.time()
    processing_time = end_time - start_time
    print(f"The script took {processing_time} seconds to complete.")

if __name__ == "__main__":
    main()

For hptripid polygon in Bike mode: M11814AC.03.02, POI count= 14467
For hptripid polygon in Bike mode: M12326MN.01.01, POI count= 4287
For hptripid polygon in Bike mode: M14307HC.01.03, POI count= 2718
For hptripid polygon in Bike mode: M14417TD.03.01, POI count= 1924
For hptripid polygon in Bike mode: M13841WB.01.01, POI count= 810
For hptripid polygon in Bike mode: M14307HC.01.05, POI count= 609
For hptripid polygon in Bike mode: M13142RR.01.02, POI count= 589
For hptripid polygon in Bike mode: M13841WB.01.02, POI count= 449
For hptripid polygon in Bike mode: M14970FH.01.02, POI count= 296
For hptripid polygon in Bike mode: M14970FH.01.01, POI count= 266
For hptripid polygon in Bike mode: M15142US.02.07, POI count= 230
For hptripid polygon in Bike mode: M13142RR.01.01, POI count= 205
For hptripid polygon in Bike mode: M14307HC.02.02, POI count= 200
For hptripid polygon in Bike mode: M14307HC.02.01, POI count= 189
For hptripid polygon in Bike mode: M11814AC.03.01, POI count= 141
For h